In [1]:
import pandas as pd
import requests as re
from io import BytesIO
from os import path, makedirs

# Produção de habitação de interesse social

Vários indicadores do GT Urbanismo são relacionados à produção e entrega de unidades de habitação de interesse social. O formulário 15 é relacionado ao orçamento utilizado na produção de unidades de habitação de interesse social. O primeiro indicador físico está no formulário 18 e diz respeito à meta de provimento de moradias do PdM 2021-2024. O formulário 20 é relacionado a uma iniciativa específica do PdM, mas os dados de execução estão disponíveis apenas no relatório da Função Habitação. Por último, os formulários 19 e 21 são relacionado, mas não possuem meta no PdM, apenas nos ODS da Agenda 2030.

Posteriormente, um novo documento apresentou várias informações diferentes relacionadas ao orçamento da função habitação, onde está incluso o Programa 3002. Portanto, vamos manter todo o orçamento da função habitação e apresentar os diferentes recortes no Qlik Sense.

## Orçamento previsto/liquidado no Programa 3002

Vamos começar coletando os dados orçamentários do site de execução orçamentária da Secretaria da Fazenda.

In [2]:
def download_orcamento(url_orcamento):
    df_orcamento = pd.read_csv(url_orcamento,
                               sep=';',
                               encoding='latin1',
                               dtype=str)
    for col in [col for col in df_orcamento.columns if 'Vl' in col]:
        df_orcamento[col] = df_orcamento[col].str.replace('.', '')
        df_orcamento[col] = df_orcamento[col].str.replace(',', '.')
        df_orcamento[col] = df_orcamento[col].str.replace(',', '.').astype(float)
    if 'DataExtracao' in df_orcamento.columns:
        df_orcamento.loc[:, 'DataExtracao'] = pd.to_datetime(df_orcamento['DataExtracao'], format='%d/%m/%Y')
    df_orcamento = df_orcamento.loc[df_orcamento['Cd_Funcao']=='16']
    return df_orcamento

In [3]:
df_orcamento = download_orcamento('https://orcamento.sf.prefeitura.sp.gov.br/orcamento/uploads/2025/basedadosexecucaoConsolidadosComRestos_0825.csv')
df_orcamento = df_orcamento[
    df_orcamento['Cd_Exercicio'].isin([f'202{i}' for i in range(1, 5)])]
df_orcamento

,DataInicial,DataFinal,Cd_Exercicio,Cd_AnoExecucao,Cd_Dotacao_Id,Administracao,Cd_Orgao,Sigla_Orgao,Ds_Orgao,Cd_Unidade,...,Vl_CongeladoLiquido,Disponivel,Vl_ReservadoLiquido,Vl_Empenhado,Empenhado_Anulado,Vl_EmpenhadoLiquido,Vl_Liquidado,Vl_Pago,Saldo_Dotacao,DataExtracao
146725,01/01/2021,31/12/2021,2021,2021,147195,Direta,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,1000.0,0,0.0,0.0,0,0.00,0.00,0.00,0,2022-12-31 00:00:00
146726,01/01/2021,31/12/2021,2021,2021,147194,Direta,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,1000.0,0,0.0,0.0,0,0.00,0.00,0.00,0,2022-12-31 00:00:00
146727,01/01/2021,31/12/2021,2021,2021,147196,Direta,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,1000.0,0,0.0,0.0,0,0.00,0.00,0.00,0,2022-12-31 00:00:00
146728,01/01/2021,31/12/2021,2021,2021,147198,Direta,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,1000.0,0,0.0,0.0,0,0.00,0.00,0.00,0,2022-12-31 00:00:00
146729,01/01/2021,31/12/2021,2021,2021,147197,Direta,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,1000.0,0,0.0,0.0,0,0.00,0.00,0.00,0,2022-12-31 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186923,01/01/2025,31/08/2025,2024,2025,180764,Indireta,91,FMH,Fundo Municipal de Habitação,10,...,0.0,0,0.0,0.0,"8865221,58",-8865221.58,4178723.14,4178723.14,0,2025-08-15 00:00:00
186924,01/01/2025,31/08/2025,2024,2025,164693,Indireta,91,FMH,Fundo Municipal de Habitação,10,...,0.0,0,0.0,0.0,"1218657,08",-1218657.08,106449.52,106449.52,0,2025-08-15 00:00:00
186925,01/01/2025,31/08/2025,2024,2025,164694,Indireta,91,FMH,Fundo Municipal de Habitação,10,...,0.0,0,0.0,0.0,"1726137,54",-1726137.54,400472.79,400472.79,0,2025-08-15 00:00:00
186926,01/01/2025,31/08/2025,2024,2025,164700,Indireta,91,FMH,Fundo Municipal de Habitação,10,...,0.0,0,0.0,0.0,"676470,38",-676470.38,0.00,0.00,0,2025-08-15 00:00:00


## Orçamento regionalizado no Programa 3002

O orçamento é regionalizado apenas na liquidação, então não é possível obter o orçamento previsto por subprefeitura, mas é possível obter o orçamento liquidado.

In [4]:
import numpy as np
def download_orcamento_r(url_orcamento_r):
    df_orcamento_r = pd.read_csv(url_orcamento_r,
                                 sep=';',
                                 encoding='latin1',
                                 dtype=str)
    df_orcamento_r['VALOR_DETALHAMENTO_AÇÃO'] = df_orcamento_r['VALOR_DETALHAMENTO_AÇÃO'].str.replace('R$', '')
    df_orcamento_r['VALOR_DETALHAMENTO_AÇÃO'] = df_orcamento_r['VALOR_DETALHAMENTO_AÇÃO'].str.replace('(', '')
    df_orcamento_r['VALOR_DETALHAMENTO_AÇÃO'] = df_orcamento_r['VALOR_DETALHAMENTO_AÇÃO'].str.replace(')', '')
    df_orcamento_r['VALOR_DETALHAMENTO_AÇÃO'] = df_orcamento_r['VALOR_DETALHAMENTO_AÇÃO'].str.replace('-', '')
    df_orcamento_r['VALOR_DETALHAMENTO_AÇÃO'] = df_orcamento_r['VALOR_DETALHAMENTO_AÇÃO'].str.replace('.', '')
    df_orcamento_r['VALOR_DETALHAMENTO_AÇÃO'] = df_orcamento_r['VALOR_DETALHAMENTO_AÇÃO'].str.replace(',', '.')
    df_orcamento_r['VALOR_DETALHAMENTO_AÇÃO'] = df_orcamento_r['VALOR_DETALHAMENTO_AÇÃO'].str.strip()
    df_orcamento_r['VALOR_DETALHAMENTO_AÇÃO'] = df_orcamento_r['VALOR_DETALHAMENTO_AÇÃO'].replace('', np.nan)
    df_orcamento_r['VALOR_DETALHAMENTO_AÇÃO'] = df_orcamento_r['VALOR_DETALHAMENTO_AÇÃO'].astype(float)
    # if 'DATA_EXTRAÇÃO' in df_orcamento_r.columns:
        # df_orcamento_r.loc[:, 'DATA_EXTRAÇÃO'] = pd.to_datetime(df_orcamento_r['DATA_EXTRAÇÃO'], format='%d/%m/%Y')
    df_orcamento_r = df_orcamento_r.loc[df_orcamento_r['CÓDIGO_FUNÇÃO']=='16']
    return df_orcamento_r

In [5]:
urls_orcamento_r = ['https://orcamento.sf.prefeitura.sp.gov.br/orcamento/uploads/2021/EXECU%C3%87%C3%83O_DA_2021.csv',
                    'https://orcamento.sf.prefeitura.sp.gov.br/orcamento/uploads/2022/EXECU%C3%87%C3%83O_DA_2022_v2.csv',
                    'https://orcamento.sf.prefeitura.sp.gov.br/orcamento/uploads/2023/basedadosDA_1223.csv',
                    'https://orcamento.sf.prefeitura.sp.gov.br/orcamento/uploads/2024/basedadosDA_1224.csv']

df_orcamento_r = pd.concat([download_orcamento_r(url) for url in urls_orcamento_r], ignore_index=True)

df_orcamento_r

,COD_EMPRESA_PMSP,COD_EMPENHO,ANO_EMPENHO,CÓDIGO_NLP,ANO_LIQUIDAÇÃO,DATA_LIQUIDAÇÃO,CÓDIGO_ÓRGÃO,SIGLA_ÓRGÃO,DESCRIÇÃO_ÓRGÃO,CÓDIGO_UNIDADE,...,REGIÃO,SUBPREFEITURA,DISTRITO,TIPO_REGIONALIZAÇÃO,VALOR_DETALHAMENTO_AÇÃO,DATA_EXTRAÇÃO,CÓDIGO_EXERCÍCIO_FONTE,CÓDIGO_DESTINAÇÃO_RECURSO,CÓDIGO_VÍNCULO_PMSP,CÓDIGO_TIPO_CRÉDITO_ORÇAMENTÁRIO
0,01,239,2021,4965,2021,18/01/2021 00:00,14,SEHAB,Secretaria Municipal de Habitação,10,...,NaN,NaN,NaN,NaN,NaN,12/07/2023 03:25,NaN,NaN,NaN,NaN
1,01,239,2021,9490,2022,21/01/2022 00:00,14,SEHAB,Secretaria Municipal de Habitação,10,...,NaN,NaN,NaN,NaN,NaN,12/07/2023 03:25,NaN,NaN,NaN,NaN
2,01,239,2021,10350,2021,26/01/2021 00:00,14,SEHAB,Secretaria Municipal de Habitação,10,...,NaN,NaN,NaN,NaN,NaN,12/07/2023 03:25,NaN,NaN,NaN,NaN
3,01,239,2021,22459,2021,05/02/2021 00:00,14,SEHAB,Secretaria Municipal de Habitação,10,...,NaN,NaN,NaN,NaN,NaN,12/07/2023 03:25,NaN,NaN,NaN,NaN
4,01,239,2021,28774,2021,15/02/2021 00:00,14,SEHAB,Secretaria Municipal de Habitação,10,...,NaN,NaN,NaN,NaN,NaN,12/07/2023 03:25,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24102,01,51229,2024,199845,2024,2024-07-25 00:00:00.0000000,98,FUNDURB,Fundo de Desenvolvimento Urbano,14,...,Sul,Subprefeitura Parelheiros,Supra-Distrital,Despesa Regionalizável,5586408.52,15/08/2025,2,759,0402,1
24103,01,68392,2024,199979,2024,2024-07-25 00:00:00.0000000,98,FUNDURB,Fundo de Desenvolvimento Urbano,14,...,Sul,Subprefeitura Capela do Socorro,Supra-Distrital,Despesa Regionalizável,4146745.21,15/08/2025,2,759,0402,1
24104,01,68392,2024,271158,2024,2024-09-30 00:00:00.0000000,98,FUNDURB,Fundo de Desenvolvimento Urbano,14,...,Sul,Subprefeitura Capela do Socorro,Supra-Distrital,Despesa Regionalizável,1436116.72,15/08/2025,2,759,0402,1
24105,01,68392,2024,271159,2024,2024-09-30 00:00:00.0000000,98,FUNDURB,Fundo de Desenvolvimento Urbano,14,...,Sul,Subprefeitura Capela do Socorro,Supra-Distrital,Despesa Regionalizável,417138.07,15/08/2025,2,759,0402,1


Vamos conferir qual o percentual do orçamento está presente na tabela de detalhamento e regionalizado a nível de subprefeitura.

In [6]:
df_orcamento_r['VALOR_DETALHAMENTO_AÇÃO'].sum()/df_orcamento['Vl_Liquidado'].sum()

np.float64(0.8714594365913676)

87,2% de detalhamento é um percentual excelente. Vamos checar a regionalização.

In [7]:
(
    df_orcamento_r
    .assign(regionalizado=df_orcamento_r['SUBPREFEITURA'].str.contains('Supra')==False)
    .groupby('regionalizado')
    ['VALOR_DETALHAMENTO_AÇÃO'].sum()/df_orcamento['Vl_Liquidado'].sum()
)

regionalizado
False    0.267666
True     0.603793
Name: VALOR_DETALHAMENTO_AÇÃO, dtype: float64

O percentual de regionalização é de 60,4%. Menor, mas ainda parece ser satisfatório. Vamos manter as informações de regionalização também.

## Cumprimento da Meta 12: Prover 49.000 moradias de interesse social

Esse indicador está disponível de forma regionalizada na página do PdM da Secretaria de Governo Municipal. 

In [8]:
url_meta_12 = 'https://prefeitura.sp.gov.br/documents/d/planejamento/relatorio-de-final-do-programa-de-metas-2021-2024_marco2025-xlsx'

resp_meta_12 = re.get(url_meta_12)
resp_meta_12.raise_for_status()
xlsx_meta_12 = BytesIO(resp_meta_12.content)
excel_meta_12 = pd.ExcelFile(xlsx_meta_12)

excel_meta_12.sheet_names

['execução', 'regionalização', 'orçamento']

In [9]:
sheet_name = 'regionalização'
df_meta_12 = pd.read_excel(xlsx_meta_12,
                   sheet_name=sheet_name)
df_meta_12

,Número da Meta,Descrição,Área,Subprefeitura,2021,2022,2023,2024
0,2,Implantar o Prontuário Eletrônico em 100% das ...,Saúde,Aricanduva,44.0,44.0,100.0,100.0
1,2,Implantar o Prontuário Eletrônico em 100% das ...,Saúde,Butantã,0.0,0.0,0.0,100.0
2,2,Implantar o Prontuário Eletrônico em 100% das ...,Saúde,Campo Limpo,100.0,100.0,100.0,100.0
3,2,Implantar o Prontuário Eletrônico em 100% das ...,Saúde,Capela do Socorro,44.0,44.0,92.0,100.0
4,2,Implantar o Prontuário Eletrônico em 100% das ...,Saúde,Casa Verde,54.0,69.0,100.0,100.0
...,...,...,...,...,...,...,...,...
960,84,Implantar quatro unidades da Escola Municipal ...,Cultura,Butantã,0.0,0.0,1.0,1.0
961,84,Implantar quatro unidades da Escola Municipal ...,Cultura,Freguesia/Brasilândia,0.0,1.0,1.0,1.0
962,84,Implantar quatro unidades da Escola Municipal ...,Cultura,Itaim Paulista,0.0,0.0,1.0,1.0
963,84,Implantar quatro unidades da Escola Municipal ...,Cultura,Parelheiros,0.0,0.0,0.0,1.0


In [10]:
df_meta_12 = df_meta_12[df_meta_12['Número da Meta']==12]
df_meta_12

,Número da Meta,Descrição,Área,Subprefeitura,2021,2022,2023,2024
151,12,Prover 49.000 moradias de interesse social.,Habitação,Butantã,216.0,546.0,3081.0,7261.0
152,12,Prover 49.000 moradias de interesse social.,Habitação,Campo Limpo,1316.0,1748.0,1748.0,2161.0
153,12,Prover 49.000 moradias de interesse social.,Habitação,Capela do Socorro,562.0,2040.0,2124.0,2140.0
154,12,Prover 49.000 moradias de interesse social.,Habitação,Casa Verde,0.0,108.0,128.0,128.0
155,12,Prover 49.000 moradias de interesse social.,Habitação,Cidade Ademar,0.0,8.0,78.0,78.0
156,12,Prover 49.000 moradias de interesse social.,Habitação,Cidade Tiradentes,50.0,50.0,347.0,507.0
157,12,Prover 49.000 moradias de interesse social.,Habitação,Ermelino Matarazzo,0.0,0.0,0.0,266.0
158,12,Prover 49.000 moradias de interesse social.,Habitação,Freguesia/Brasilândia,228.0,228.0,228.0,502.0
159,12,Prover 49.000 moradias de interesse social.,Habitação,Guaianases,0.0,968.0,1122.0,1322.0
160,12,Prover 49.000 moradias de interesse social.,Habitação,Ipiranga,1694.0,1965.0,2205.0,2685.0


## Produção de habitação de interesse social

O indicador do formulário 19 não está ligado diretamente a nenhuma meta específica do PdM, mas está em consonância com o indicador 11.01.03 dos ODS da Agenda 2030. Os dados desse indicador estão disponíveis no [Observasampa](https://observasampa.prefeitura.sp.gov.br/).

In [11]:
url_his = 'https://observasampa.prefeitura.sp.gov.br/arquivo.php?cd_indicador=285'
df_his = pd.read_csv(url_his,
                     sep=';',
                     decimal=',',
                     encoding='utf8')
df_his

,Unnamed: 0,região,nivel_regional,indicador,2017,2018,2019,2020,2021,2022,2023,2024
0,0,São Paulo,Município,11.01.03 Número de Unidades Habitacionais entr...,1982.0,2263.0,5153.0,5697.0,3282.0,2400.0,755.0,1145.0
1,0,Pirituba-Jaraguá,Subprefeitura,11.01.03 Número de Unidades Habitacionais entr...,NaN,NaN,NaN,592.0,0.0,0.0,0.0,5.0
2,1,Cidade Tiradentes,Subprefeitura,11.01.03 Número de Unidades Habitacionais entr...,NaN,NaN,NaN,39.0,50.0,0.0,0.0,2.0
3,2,Ermelino Matarazzo,Subprefeitura,11.01.03 Número de Unidades Habitacionais entr...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0
4,3,Parelheiros,Subprefeitura,11.01.03 Número de Unidades Habitacionais entr...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
124,91,Sapopemba,Distrito,11.01.03 Número de Unidades Habitacionais entr...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
125,92,Saúde,Distrito,11.01.03 Número de Unidades Habitacionais entr...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
126,93,Sé,Distrito,11.01.03 Número de Unidades Habitacionais entr...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
127,94,Socorro,Distrito,11.01.03 Número de Unidades Habitacionais entr...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0


In [12]:
df_his = df_his[df_his['nivel_regional']=='Subprefeitura']
df_his

,Unnamed: 0,região,nivel_regional,indicador,2017,2018,2019,2020,2021,2022,2023,2024
1,0,Pirituba-Jaraguá,Subprefeitura,11.01.03 Número de Unidades Habitacionais entr...,NaN,NaN,NaN,592.0,0.0,0.0,0.0,5.0
2,1,Cidade Tiradentes,Subprefeitura,11.01.03 Número de Unidades Habitacionais entr...,NaN,NaN,NaN,39.0,50.0,0.0,0.0,2.0
3,2,Ermelino Matarazzo,Subprefeitura,11.01.03 Número de Unidades Habitacionais entr...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0
4,3,Parelheiros,Subprefeitura,11.01.03 Número de Unidades Habitacionais entr...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
5,4,Penha,Subprefeitura,11.01.03 Número de Unidades Habitacionais entr...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
6,5,Perus,Subprefeitura,11.01.03 Número de Unidades Habitacionais entr...,NaN,NaN,NaN,32.0,0.0,48.0,110.0,1.0
7,6,Pinheiros,Subprefeitura,11.01.03 Número de Unidades Habitacionais entr...,NaN,NaN,NaN,0.0,0.0,0.0,109.0,0.0
8,7,Santana-Tucuruvi,Subprefeitura,11.01.03 Número de Unidades Habitacionais entr...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,401.0
9,8,Santo Amaro,Subprefeitura,11.01.03 Número de Unidades Habitacionais entr...,NaN,NaN,NaN,0.0,300.0,0.0,0.0,0.0
10,9,São Mateus,Subprefeitura,11.01.03 Número de Unidades Habitacionais entr...,NaN,NaN,NaN,926.0,816.0,600.0,0.0,4.0


In [13]:
df_his = df_his.loc[:, [
    'região',
    'indicador',
    '2021',
    '2022',
    '2023'
]]

df_his

,região,indicador,2021,2022,2023
1,Pirituba-Jaraguá,11.01.03 Número de Unidades Habitacionais entr...,0.0,0.0,0.0
2,Cidade Tiradentes,11.01.03 Número de Unidades Habitacionais entr...,50.0,0.0,0.0
3,Ermelino Matarazzo,11.01.03 Número de Unidades Habitacionais entr...,0.0,0.0,0.0
4,Parelheiros,11.01.03 Número de Unidades Habitacionais entr...,0.0,0.0,0.0
5,Penha,11.01.03 Número de Unidades Habitacionais entr...,0.0,0.0,0.0
6,Perus,11.01.03 Número de Unidades Habitacionais entr...,0.0,48.0,110.0
7,Pinheiros,11.01.03 Número de Unidades Habitacionais entr...,0.0,0.0,109.0
8,Santana-Tucuruvi,11.01.03 Número de Unidades Habitacionais entr...,0.0,0.0,0.0
9,Santo Amaro,11.01.03 Número de Unidades Habitacionais entr...,300.0,0.0,0.0
10,São Mateus,11.01.03 Número de Unidades Habitacionais entr...,816.0,600.0,0.0


In [14]:
df_his = df_his.melt(
        ['região', 'indicador'],
        ['2021', '2022', '2023'],
        'ano',
        'qtd_unidades'
    )

df_his

,região,indicador,ano,qtd_unidades
0,Pirituba-Jaraguá,11.01.03 Número de Unidades Habitacionais entr...,2021,0.0
1,Cidade Tiradentes,11.01.03 Número de Unidades Habitacionais entr...,2021,50.0
2,Ermelino Matarazzo,11.01.03 Número de Unidades Habitacionais entr...,2021,0.0
3,Parelheiros,11.01.03 Número de Unidades Habitacionais entr...,2021,0.0
4,Penha,11.01.03 Número de Unidades Habitacionais entr...,2021,0.0
...,...,...,...,...
91,Itaim Paulista,11.01.03 Número de Unidades Habitacionais entr...,2023,0.0
92,Itaquera,11.01.03 Número de Unidades Habitacionais entr...,2023,0.0
93,Jabaquara,11.01.03 Número de Unidades Habitacionais entr...,2023,0.0
94,Jaçanã-Tremembé,11.01.03 Número de Unidades Habitacionais entr...,2023,0.0


## Número de termos de Permissão de Uso (TPU) emitidos em nome da mulher da familia

Esse indicador possui é um exemplo da transversalidade de gênero, que integra a perspectiva de gênero na construção da politica pública voltada à provisão de habitação de interesse social. Os dados desse indicador estão disponíveis, de maneira não regionalizada, no [Observasampa](https://observasampa.prefeitura.sp.gov.br/), no indicador 05.0a.04. Posteriormente, os dados regionalizados estarão presentes no relatório da Função Habitação.

In [15]:
url_tpu = 'https://observasampa.prefeitura.sp.gov.br/arquivo.php?cd_indicador=662'
df_tpu = pd.read_csv(url_tpu,
                     sep=';',
                     decimal=',',
                     encoding='utf8')
df_tpu

,Unnamed: 0,região,nivel_regional,indicador,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,0,São Paulo,Município,05.0a.04 Número de termos de Permissão de Uso ...,327.0,234.0,386.0,544.0,192.0,300.0,285.0,293.0,384.0,458.0


In [16]:
df_tpu = df_tpu.loc[:, [
    'indicador',
    '2021',
    '2022',
    '2023'
]]

df_tpu

,indicador,2021,2022,2023
0,05.0a.04 Número de termos de Permissão de Uso ...,285.0,293.0,384.0


In [17]:
df_tpu = df_tpu.melt(
        ['indicador'],
        ['2021', '2022', '2023'],
        'ano',
        'qtd_termos'
    )

df_tpu

,indicador,ano,qtd_termos
0,05.0a.04 Número de termos de Permissão de Uso ...,2021,285.0
1,05.0a.04 Número de termos de Permissão de Uso ...,2022,293.0
2,05.0a.04 Número de termos de Permissão de Uso ...,2023,384.0


# Exportando os arquivos

Neste notebook, vamos apenas salvar os arquivos extraídos na pasta de entrada de dados.

In [18]:
output_dir = path.join('dados', 'urbanismo')

if not path.exists(output_dir):
    makedirs(output_dir)

for c, df in [('orcamento_habitacao_original', df_orcamento),
               ('orcamento_regionalizado_habitacao_original', df_orcamento_r),
               ('pdm_meta_12_original', df_meta_12),
               ('his_entregue_original', df_his),
               ('tpu_emitido_original', df_tpu)]:
    filename=path.join(output_dir, c)
    df.to_csv(f'{filename}.csv',
              sep=';',
              decimal=',',
              encoding='utf8',
              index=False
              )